In [1]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_diagonal_wavelet_axial_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,9024206.41,105.18,-144.75,34.93,-11.15,-3.75,-0.33,0.09
3,105.18,2839506.50,-75.18,-0.77,14.48,3.72,0.19,0.01
4,-144.75,-75.18,819023.16,36.38,1.23,-1.03,-0.11,0.03
5,34.93,-0.77,36.38,152412.25,-6.28,-0.21,0.07,-0.01
6,-11.15,14.48,1.23,-6.28,55937.57,-0.47,0.03,0.01
7,-3.75,3.72,-1.03,-0.21,-0.47,3509.46,0.01,0.00
8,-0.33,0.19,-0.11,0.07,0.03,0.01,100.82,-0.00
9,0.09,0.01,0.03,-0.01,0.01,0.00,-0.00,2.16


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00002,-0.00005,0.00003,-0.00002,-0.00002,-0.00001,0.00002
3,0.00002,1.00000,-0.00005,-0.00000,0.00004,0.00004,0.00001,0.00000
4,-0.00005,-0.00005,1.00000,0.00010,0.00001,-0.00002,-0.00001,0.00002
5,0.00003,-0.00000,0.00010,1.00000,-0.00007,-0.00001,0.00002,-0.00001
6,-0.00002,0.00004,0.00001,-0.00007,1.00000,-0.00003,0.00001,0.00002
7,-0.00002,0.00004,-0.00002,-0.00001,-0.00003,1.00000,0.00002,0.00001
8,-0.00001,0.00001,-0.00001,0.00002,0.00001,0.00002,1.00000,-0.00004
9,0.00002,0.00000,0.00002,-0.00001,0.00002,0.00001,-0.00004,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

285.0092689068232

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.02040317e+13 9.87730937e+11 7.97674402e+10 2.67674830e+09
 3.32093672e+08 1.17262652e+06 7.26224568e+02 2.81265587e-32]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,9.986829e-01,-4.961278e-02,-0.012837,-0.002361,-0.000869,-0.000055,-0.000002,-2.404470e-08
1,4.884195e-02,9.973554e-01,-0.052974,-0.008926,-0.003260,-0.000203,-0.000006,-1.241462e-07
2,1.529597e-02,5.184859e-02,0.997682,-0.038942,-0.013792,-0.000862,-0.000025,-4.952786e-07
3,3.243212e-03,1.034198e-02,0.036743,0.993714,-0.105051,-0.005491,-0.000157,-3.429279e-06
4,1.585124e-03,5.030989e-03,0.017508,0.104272,0.994144,-0.021705,-0.000619,-1.315406e-05
5,1.300917e-04,4.106991e-04,0.001430,0.007682,0.020983,0.999646,-0.014386,-3.069259e-04
6,5.625845e-06,1.765325e-05,0.000062,0.000330,0.000900,0.014359,0.999668,-2.138965e-02
7,2.299410e-07,7.537714e-07,0.000003,0.000014,0.000038,0.000614,0.021383,9.997712e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0013171429551341873,
 0.002644567181673718,
 0.0023175242386851425,
 0.006286489369838444,
 0.0058563752616088705,
 0.00035434702572978427,
 0.00033235883908300234,
 0.00022883194096323667]